## Send Sensor Telemtry to RTI
Stream data from the SensorData.csv file to RTI for processing.

In [ ]:
# Install the required package
%pip install azure-eventhub

In [2]:
# import required libraries
import random
import time
import json
import os
from azure.eventhub import EventHubConsumerClient, EventHubProducerClient,EventData
import csv
import requests
import time

StatementMeta(, be711a4a-5123-4f35-a65e-7aadddafc4b1, 10, Finished, Available, Finished)

### Test the Data Connections 

In [ ]:
# Test Cell to ensure the file is accessible and has data - Optional to run.

# Define the path to the CSV file in the Lakehouse 
# UPDATE WITH YOUR LAKEHOUSE
file_path = "abfss://<###YOUR WORKSPACE###>@onelake.dfs.fabric.microsoft.com/ZavaLakehouse.Lakehouse/Files/RawData/SensorData/SensorData.csv"

# Load the CSV into a DataFrame
df = spark.read.option("header", True).option("inferSchema", True).csv(file_path)

# Show the first few rows
df.show()


StatementMeta(, be711a4a-5123-4f35-a65e-7aadddafc4b1, 12, Finished, Available, Finished)

+-------------------+-----------+----+--------+---------+
|          timestamp|sensorValue|unit|sensorId|sessionID|
+-------------------+-----------+----+--------+---------+
|2023-01-01 00:00:00|       50.5| kPa| SEN0003| S0000150|
|2023-01-01 00:00:01|      85.32| kPa| SEN0027| S0000150|
|2023-01-01 00:00:02|      78.35| kPa| SEN0051| S0000150|
|2023-01-01 00:00:03|      37.18| kPa| SEN0075| S0000150|
|2023-01-01 00:00:04|      61.45| kPa| SEN0099| S0000150|
|2023-01-01 00:00:05|      61.16| kPa| SEN0123| S0000150|
|2023-01-01 00:00:06|      65.89| kPa| SEN0003| S0000150|
|2023-01-01 00:00:07|      88.39| kPa| SEN0027| S0000150|
|2023-01-01 00:00:08|      71.91| kPa| SEN0051| S0000150|
|2023-01-01 00:00:09|      54.37| kPa| SEN0075| S0000150|
|2023-01-01 00:00:10|      66.26| kPa| SEN0099| S0000150|
|2023-01-01 00:00:11|       45.3| kPa| SEN0123| S0000150|
|2023-01-01 00:00:12|      51.54| kPa| SEN0003| S0000150|
|2023-01-01 00:00:13|       33.3| kPa| SEN0027| S0000150|
|2023-01-01 00

### Start the data generation below

In [ ]:
# Step 1 Setup Connection Info
connection_string = "Endpoint=sb://<###YOUR RTI ENDPOINT###>.servicebus.windows.net/;";
producer = EventHubProducerClient.from_connection_string(connection_string)

# Step 2: Read CSV file from Lakehouse Files

file_path = "abfss://<###YOUR WORKSPACE###>@onelake.dfs.fabric.microsoft.com/ZavaLakehouse.Lakehouse/Files/RawData/SensorData/SensorDataShoe.csv"
df = spark.read.option("header", True).option("inferSchema", True).csv(file_path)

# Step 2: Convert DataFrame to JSON rows
json_rows = df.toJSON().collect()

In [ ]:
# Step 4: Send 10 rows every 1 second and loop back to the beginning
with producer:
   while True:
     for i in range(0, len(json_rows), 10):
          batch = json_rows[i:i+10]
          event_data_batch = [EventData(row) for row in batch]
          producer.send_batch(event_data_batch)
          # print(f"Sent batch {i//10 + 1}")
          time.sleep(1)